to be run on Windows

## Rides data

In [ ]:
import pandas as pd
import urllib.request
import gzip
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

In [ ]:
file_name = 'green_tripdata_2019-01.csv.gz'
url = f'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/{file_name}'

In [ ]:
urllib.request.urlretrieve(url, file_name)

In [ ]:
with gzip.open(file_name) as f:

    df_rides = pd.read_csv(f)

In [ ]:
df_rides['lpep_pickup_datetime'] = pd.to_datetime(df_rides['lpep_pickup_datetime'])
df_rides['lpep_dropoff_datetime'] = pd.to_datetime(df_rides['lpep_dropoff_datetime'])

In [ ]:
sql_create_rides = pd.io.sql.get_schema(df_rides.reset_index(), 'fct_green_trips')

In [ ]:
conn = psycopg2.connect(
host = 'localhost',
database = 'ny_taxi',
user = 'root',
password = 'root'
)

cursor = conn.cursor()

cursor.execute(sql_create_rides)

conn.commit()

## Zones

In [ ]:
file_name_zones = 'taxi+_zone_lookup.csv'
url_zones = f'https://s3.amazonaws.com/nyc-tlc/misc/{file_name_zones}'

In [ ]:
urllib.request.urlretrieve(url_zones, file_name_zones)

In [ ]:
df_zones = pd.read_csv('taxi+_zone_lookup.csv')

In [ ]:
sql_create_zones = pd.io.sql.get_schema(df_zones.reset_index(), 'dim_zones')

In [ ]:
cursor.execute(sql_create_zones)

conn.commit()

In [ ]:
url = URL.create(
    drivername="postgresql",
    username="root",
    host="localhost",
    database="ny_taxi",
    password = 'root'
)

engine = create_engine(url)

In [ ]:
df_zones.to_sql('dim_zones', engine, 'public', 'append')